In [1]:
import os
import pandas as pd
import parse
from transliterate import translit

labels = "/home/peterr/macocu/Varieties/BCMS/Denis_hrsrTweets.txt"
additional_labels =  "/home/peterr/macocu/Varieties/BCMS/Denis_hrsrTweets_additional.txt"


In [2]:
p = parse.compile("users/{username}{whitespace:s}{language}\n")
userdict = dict()
with open(labels, "r") as labelfile:
    for line in labelfile.readlines():
        line = translit(line, "sr", reversed=True)
        found = p.parse(line)
        userdict[found["username"]] = found["language"]
with open(additional_labels, "r") as labelfile:
    for line in labelfile.readlines():
        #line = translit(line, "sr", reversed=True)
        found = p.parse(line)
        userdict[found["username"]] = found["language"]

twitter_data_dir = "/home/peterr/macocu/Varieties/BCMS/users"
datafiles = os.listdir(twitter_data_dir)

missing = [filename for filename in datafiles if filename not in userdict.keys()]
len(missing)

386

In [60]:
from bs4 import BeautifulSoup
from typing import List
def read_user_tweets(filename: str) -> List[str]:
    with open(filename, "r") as f:
        content = "<data>"+f.read()+"<\data>"
    data = BeautifulSoup(content, "xml")
    tweets = data.findAll("tweet")
    return [tweet.find("text").text for tweet in tweets]
import re
def remove_mentions(tweet: str) -> str:
    re_pattern = r"(^|[^@\w])@(\w{1,15})\b"
    return re.sub(re_pattern, " ", tweet)
def remove_URL(tweet: str) -> str:
    re_pattern = r"(https?:\/\/|\b)(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-z]{2,4}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)"
    return re.sub(re_pattern, " ", tweet)
def remove_hashtags(tweet: str) -> str:
    re_pattern = r"#\w*[a-zA-Z]\w*"
    return re.sub(re_pattern, " ", tweet)
def remove_retweet_tag(tweet: str) -> str:
    re_pattern = r"RT @(\w{1,15}): "
    return re.sub(re_pattern, " ", tweet)
def to_latin(tweet: str) -> str:
    for lang in ["sr", "mk"]:
        tweet = translit(tweet, lang, reversed=True)
    return tweet
def remove_nonlatin_chars(tweet: str) -> str:
    import regex
    # result = regex.sub(u'[^\p{Latin}]', u'', tweet)
    # return " ".join([regex.sub(u'[^\p{Latin}]', u'', word) for word in tweet.split(" ")])
    import re
    result = re.sub(u'[^\x00-\x7F\x80-\xFF\u0100-\u017F\u0180-\u024F\u1E00-\u1EFF]', u'', tweet)
    return result 

In [64]:
remove_nonlatin_chars(":D hello.  ÷÷永타ﬀ(👌🍎☉")

':D hello.  ÷÷('

In [62]:
dataset = dict()
for user, language in userdict.items():
    filename = os.path.join(twitter_data_dir, user)
    tweets = read_user_tweets(filename)

    def preprocess(s): return remove_nonlatin_chars(remove_hashtags(remove_URL(
        remove_mentions(remove_retweet_tag(to_latin(s))))))
    corrected_tweets = list(map(preprocess, tweets))
    corrected_tweets = [t for t in corrected_tweets if len(t) > 0]
    dataset[user] = {"language": language, "tweets": corrected_tweets}


In [63]:
dataset[user]

{'language': 'hr',
 'tweets': ['  Nadam se uskoro',
  'Bravo Rijeka. Čestitam dečkima na plasmanu u  play off.',
  '   Zračna luka je u većinskom vlasništvu države, a za autobusni kolodvor se treba strpiti do 1.  natječaja za EU fondove',
  '    Reći ću samo he he!',
  'Još se možete uključiti u raspravu o strategiji razvoja Rijeke do 2020.  ',
  '  Ne razumijem što je pjesnik želio reći',
  '    Seveda, tim več ko prihaja iz Maribora kot moj oče',
  'Vatrena atmosfera na Kantridi uz odlicnu igru i navijanje. Ajmo Rijeka!  ',
  '  Hoće. Sutra će se znati mjesto, ali prijenos je dogovoren',
  'RT Tocnije nocas u 00:30 : Docek nogometasa Rijeke u 12.30 na Trgu 128. Brigade kod Bonavie',
  '  00.30',
  '  Majstore nikad nisam drzao govore na doceku sportasa a na Kantridu sam odlazio i kad je bila prazna a Rijeka bila pred ispadanjem',
  '  You are allwys wellcome. Just sand a massage',
  'Docek nogometasa Rijeke u 12.30 na Trgu 128. Brigade kod Bonavie',
  '  Thank you Chris',
  'Bravo Ri

In [67]:
import pickle
with open("/home/peterr/macocu/taskB/data/final/twitter_full_ds.pickle", "wb") as f:
    pickle.dump(dataset, f)

# Analysis

This was performed before removal on non-latin characters and saved, the rerun is just for confirmation

In [68]:
set_of_all_chars = set()
for user, payload in dataset.items():
    for tweet in payload["tweets"]:
        set_of_all_chars = set_of_all_chars.union(set(tweet))

In [69]:
len(set_of_all_chars)

223

In [70]:
d = dict()
for user, payload in dataset.items():
    for tweet in payload["tweets"]:
        for char in tweet:
            d[char] = d.get(char, 0) + 1
d

{' ': 4230221,
 'H': 32622,
 'a': 2112103,
 'h': 161132,
 '!': 67666,
 'i': 1458933,
 'j': 644650,
 'z': 270093,
 'p': 406328,
 'e': 1603953,
 'r': 737063,
 'S': 91332,
 'n': 948684,
 'o': 1447075,
 'K': 56630,
 'u': 656502,
 '&': 1866,
 'I': 92190,
 'g': 228712,
 ',': 180555,
 'l': 605227,
 'k': 540598,
 't': 764076,
 's': 856224,
 'm': 700266,
 'd': 657227,
 "'": 29971,
 '«': 37,
 'v': 509727,
 '"': 37481,
 'b': 263254,
 'P': 60764,
 'V': 40711,
 'y': 44501,
 'c': 251687,
 ':': 86490,
 'N': 82819,
 'f': 83313,
 '.': 311229,
 'R': 46074,
 'L': 39148,
 ')': 58608,
 'x': 8508,
 'B': 46640,
 'D': 71900,
 'M': 69630,
 'ć': 58231,
 '|': 1646,
 'A': 124288,
 'w': 45315,
 'T': 64741,
 '-': 44950,
 '\n': 36009,
 'C': 27527,
 'W': 7077,
 'č': 68936,
 'Š': 9433,
 '1': 26840,
 'G': 23776,
 'O': 85859,
 'F': 15609,
 '2': 23819,
 '8': 5548,
 '6': 5825,
 '3': 25872,
 '?': 45725,
 'š': 110644,
 'ž': 48651,
 'J': 48953,
 '5': 10718,
 'E': 77368,
 'U': 41251,
 ';': 6072,
 'Q': 622,
 'đ': 13334,
 '/': 

In [42]:
import matplotlib.pyplot as plt

labels = list(d.keys())
heigths = list(d.values())

import pandas as pd

df = pd.DataFrame(data={"character": labels, "counts": heigths})
df = df.sort_values(by="counts", ascending=False)

# df.to_csv("8_counts_of_original_tweet_characters.csv", index=False)